# SMIB Steady state SP vs DP

In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
import dpsimpy

#%matplotlib widget

## DP Simulation

### Parametrization

In [ ]:
# Power System
V_nom = 230e3

# Generator
nom_power = 500e6
nom_ph_ph_volt_RMS = 22e3
nom_freq = 60
nom_omega = nom_freq * 2 * math.pi
H = 5
Xpd = 0.31
Rs = 0.003 * 0
D=1.5

# Initialization parameters
init_mech_power = 300e6
init_active_power = 300e6
set_point_voltage = nom_ph_ph_volt_RMS * 1.05

# Transformer
t_ratio = V_nom / nom_ph_ph_volt_RMS

# PiLine parameters calculated from CIGRE Benchmark system
line_resistance = 6.7
line_inductance = 47.0/nom_omega
line_capacitance = 3.42e-4/nom_omega
line_conductance = 0

# Parameters for powerflow initialization
# Slack voltage: 1pu
V_slack = V_nom

# Switch to trigger fault at generator terminal
switch_open = 1e6
switch_closed = 0.1

# Simulation parameters
sim_name = "DP_SynGenTrStab_SMIB_SteadyState"
final_time = 10
time_step = 0.001
cmd_Inertia = 1.0
cmd_Damping = 1.0

### Powerflow for Initialization

In [ ]:
sim_name_pf = sim_name + "_PF"
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)
time_step_pf = final_time
final_time_pf = final_time + time_step_pf

# Components
n1_pf = dpsimpy.sp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode("n2", dpsimpy.PhaseType.Single)

# Synchronous generator ideal model
gen_pf = dpsimpy.sp.ph1.SynchronGenerator("SynGen", dpsimpy.LogLevel.debug)
gen_pf.set_parameters(rated_apparent_power=nom_power, rated_voltage=nom_ph_ph_volt_RMS,
                        set_point_active_power=init_active_power,
                        set_point_voltage=set_point_voltage*t_ratio,
                        powerflow_bus_type=dpsimpy.PowerflowBusType.PV)
gen_pf.set_base_voltage(V_nom)
gen_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.PV)

# Grid bus as Slack
extnet_pf = dpsimpy.sp.ph1.NetworkInjection("Slack", dpsimpy.LogLevel.debug)
extnet_pf.set_parameters(V_slack)
extnet_pf.set_base_voltage(V_nom)
extnet_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

# Line
line_pf = dpsimpy.sp.ph1.PiLine("PiLine", dpsimpy.LogLevel.debug)
line_pf.set_parameters(line_resistance, line_inductance, line_capacitance, line_conductance)
line_pf.set_base_voltage(V_nom)

# Topology
gen_pf.connect([n1_pf])
line_pf.connect([n1_pf, n2_pf])
extnet_pf.connect([n2_pf])
system_pf = dpsimpy.SystemTopology(nom_freq, [n1_pf, n2_pf], [gen_pf, line_pf, extnet_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute("v1", "v", n1_pf)
logger_pf.log_attribute("v2", "v", n2_pf)

# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()

### Dynamic simulation

In [ ]:
sim_name_dp = sim_name + "_DP" 
dpsimpy.Logger.set_log_dir("logs/"+sim_name_dp)

# Nodes
n1_dp = dpsimpy.dp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_dp = dpsimpy.dp.SimNode("n2", dpsimpy.PhaseType.Single)

# Components
gen_dp = dpsimpy.dp.ph1.SynchronGeneratorTrStab("SynGen", dpsimpy.LogLevel.debug)
# Xpd is given in p.u of generator base at transfomer primary side and should be transformed to network side
gen_dp.set_standard_parameters_PU(
    nom_power=nom_power, nom_volt = nom_ph_ph_volt_RMS, nom_freq=nom_freq,
    Xpd = Xpd*t_ratio**2, inertia=cmd_Inertia*H, Rs=Rs, D=cmd_Damping*D)

# Grid bus as Slack
extnet_dp = dpsimpy.dp.ph1.NetworkInjection("Slack", dpsimpy.LogLevel.debug)
extnet_dp.set_parameters(V_slack)

# Line
line_dp = dpsimpy.dp.ph1.PiLine("PiLine", dpsimpy.LogLevel.debug)
line_dp.set_parameters(line_resistance, line_inductance, line_capacitance, line_conductance)

# Topology
gen_dp.connect([n1_dp])
line_dp.connect([n1_dp, n2_dp])
extnet_dp.connect([n2_dp])
system_dp = dpsimpy.SystemTopology(nom_freq, [n1_dp, n2_dp], [gen_dp, line_dp, extnet_dp])


# Initialization of dynamic topology
system_dp.init_with_powerflow(system_pf)

# Logging
logger_dp = dpsimpy.Logger(sim_name_dp)
logger_dp.log_attribute("v1", "v", n1_dp)
logger_dp.log_attribute("v2", "v", n2_dp)

logger_dp.log_attribute("v_line", "v_intf", line_dp)
logger_dp.log_attribute("i_line", "i_intf", line_dp)

logger_dp.log_attribute("v_slack", "v_intf", extnet_dp)
logger_dp.log_attribute("i_slack", "i_intf", extnet_dp)

logger_dp.log_attribute("Ep", "Ep", gen_dp)
logger_dp.log_attribute("v_gen", "v_intf", gen_dp)
logger_dp.log_attribute("i_gen", "i_intf", gen_dp)
logger_dp.log_attribute("wr_gen", "w_r", gen_dp)
logger_dp.log_attribute("delta_r_gen", "delta_r", gen_dp)
logger_dp.log_attribute("P_elec", "P_elec", gen_dp)
logger_dp.log_attribute("P_mech", "P_mech", gen_dp)

# Simulation
sim_dp = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim_dp.set_system(system_dp)
sim_dp.set_time_step(time_step)
sim_dp.set_final_time(final_time)
sim_dp.set_domain(dpsimpy.Domain.DP)
sim_dp.add_logger(logger_dp)
    
sim_dp.run()

## SP Simulation

### Parametrization

In [ ]:
# Power System
V_nom = 230e3

# Generator
nom_power = 500e6
nom_ph_ph_volt_RMS = 22e3
nom_freq = 60
nom_omega = nom_freq * 2 * math.pi
H = 5
Xpd = 0.31
Rs = 0.003 * 0
D=1.5

# Initialization parameters
init_mech_power = 300e6
init_active_power = 300e6
set_point_voltage = nom_ph_ph_volt_RMS * 1.05

# Transformer
t_ratio = V_nom / nom_ph_ph_volt_RMS

# PiLine parameters calculated from CIGRE Benchmark system
line_resistance = 6.7
line_inductance = 47.0/nom_omega
line_capacitance = 3.42e-4/nom_omega
line_conductance = 0

# Parameters for powerflow initialization
# Slack voltage: 1pu
V_slack = V_nom

# Switch to trigger fault at generator terminal
switch_open = 1e6
switch_closed = 0.1

# Simulation parameters
sim_name = "SP_SynGenTrStab_SMIB_SteadyState"
final_time = 10
time_step = 0.001
start_time_fault = 10
end_time_fault = 10.2
cmd_Inertia = 1.0
cmd_Damping = 1.0

### Powerflow for Initialization

In [ ]:
sim_name_pf = sim_name + "_PF"
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)
time_step_pf = final_time
final_time_pf = final_time + time_step_pf

# Components
n1_pf = dpsimpy.sp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_pf = dpsimpy.sp.SimNode("n2", dpsimpy.PhaseType.Single)

# Synchronous generator ideal model
gen_pf = dpsimpy.sp.ph1.SynchronGenerator("SynGen", dpsimpy.LogLevel.debug)
gen_pf.set_parameters(rated_apparent_power=nom_power, rated_voltage=nom_ph_ph_volt_RMS,
                        set_point_active_power=init_active_power,
                        set_point_voltage=set_point_voltage*t_ratio,
                        powerflow_bus_type=dpsimpy.PowerflowBusType.PV)
gen_pf.set_base_voltage(V_nom)
gen_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.PV)

# Grid bus as Slack
extnet_pf = dpsimpy.sp.ph1.NetworkInjection("Slack", dpsimpy.LogLevel.debug)
extnet_pf.set_parameters(V_slack)
extnet_pf.set_base_voltage(V_nom)
extnet_pf.modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)

# Line
line_pf = dpsimpy.sp.ph1.PiLine("PiLine", dpsimpy.LogLevel.debug)
line_pf.set_parameters(line_resistance, line_inductance, line_capacitance, line_conductance)
line_pf.set_base_voltage(V_nom)

# Topology
gen_pf.connect([n1_pf])
line_pf.connect([n1_pf, n2_pf])
extnet_pf.connect([n2_pf])
system_pf = dpsimpy.SystemTopology(nom_freq, [n1_pf, n2_pf], [gen_pf, line_pf, extnet_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute("v1", "v", n1_pf)
logger_pf.log_attribute("v2", "v", n2_pf)

# Simulation
sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(time_step_pf)
sim_pf.set_final_time(final_time_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.do_init_from_nodes_and_terminals(False)
sim_pf.add_logger(logger_pf)
sim_pf.run()

### Dynamic simulation

In [ ]:
sim_name_sp = sim_name + "_SP" 
dpsimpy.Logger.set_log_dir("logs/"+sim_name_sp)

# Nodes
n1_sp = dpsimpy.sp.SimNode("n1", dpsimpy.PhaseType.Single)
n2_sp = dpsimpy.sp.SimNode("n2", dpsimpy.PhaseType.Single)

# Components
gen_sp = dpsimpy.sp.ph1.SynchronGeneratorTrStab("SynGen", dpsimpy.LogLevel.debug)
# Xpd is given in p.u of generator base at transfomer primary side and should be transformed to network side
gen_sp.set_standard_parameters_PU(
    nom_power=nom_power, nom_volt = nom_ph_ph_volt_RMS, nom_freq=nom_freq,
    Xpd = Xpd*t_ratio**2, inertia=cmd_Inertia*H, Rs=Rs, D=cmd_Damping*D)

# Grid bus as Slack
extnet_sp = dpsimpy.sp.ph1.NetworkInjection("Slack", dpsimpy.LogLevel.debug)
extnet_sp.set_parameters(V_slack)

# Line
line_sp = dpsimpy.sp.ph1.PiLine("PiLine", dpsimpy.LogLevel.debug)
line_sp.set_parameters(line_resistance, line_inductance, line_capacitance, line_conductance)

# Switch
fault_sp = dpsimpy.sp.ph1.varResSwitch("Br_fault", dpsimpy.LogLevel.debug)
fault_sp.set_parameters(switch_open, switch_closed)
fault_sp.set_init_parameters(time_step)
fault_sp.open()

# Topology
gen_sp.connect([n1_sp])
line_sp.connect([n1_sp, n2_sp])
extnet_sp.connect([n2_sp])
system_sp = dpsimpy.SystemTopology(nom_freq, [n1_sp, n2_sp], [gen_sp, line_sp, extnet_sp])


# Initialization of dynamic topology
system_sp.init_with_powerflow(system_pf)

# Logging
logger_sp = dpsimpy.Logger(sim_name_sp)
logger_sp.log_attribute("v1", "v", n1_sp)
logger_sp.log_attribute("v2", "v", n2_sp)

logger_sp.log_attribute("v_line", "v_intf", line_sp)
logger_sp.log_attribute("i_line", "i_intf", line_sp)

logger_sp.log_attribute("v_slack", "v_intf", extnet_sp)
logger_sp.log_attribute("i_slack", "i_intf", extnet_sp)


logger_sp.log_attribute("Ep", "Ep", gen_sp)
logger_sp.log_attribute("v_gen", "v_intf", gen_sp)
logger_sp.log_attribute("i_gen", "i_intf", gen_sp)
logger_sp.log_attribute("wr_gen", "w_r", gen_sp)
logger_sp.log_attribute("delta_r_gen", "delta_r", gen_sp)
logger_sp.log_attribute("P_elec", "P_elec", gen_sp)
logger_sp.log_attribute("P_mech", "P_mech", gen_sp)

# Simulation
sim_sp = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim_sp.set_system(system_sp)
sim_sp.set_time_step(time_step)
sim_sp.set_final_time(final_time)
sim_sp.set_domain(dpsimpy.Domain.SP)
sim_sp.add_logger(logger_sp)
    
sim_sp.run()

## Results 1ph SP

In [ ]:
work_dir = 'logs/SP_SynGenTrStab_SMIB_SteadyState_SP/'
log_name = 'SP_SynGenTrStab_SMIB_SteadyState_SP'
print(work_dir + log_name + '.csv')
ts_sp1ph_TrStab_dl= rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

## Results 1ph DP

In [ ]:
work_dir = 'logs/DP_SynGenTrStab_SMIB_SteadyState_DP/' 
log_name = 'DP_SynGenTrStab_SMIB_SteadyState_DP'
print(work_dir + log_name + '.csv')
ts_dp1ph_TrStab_dl = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

## Results 3ph EMT Reference

In [ ]:
# work_dir = 'logs/EMT_SynGenTrStab_SMIB_SteadyState_EMT/'
# log_name = 'EMT_SynGenTrStab_SMIB_SteadyState_EMT'
# print(work_dir + log_name + '.csv')
# #os.path.getsize(work_dir + log_name + '.csv')
# ts_emt3ph_TrStab_dl = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

## Generator emf

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Generator emf (V)')

timestep=50e-6;
t_begin=0
t_end=20


begin_idx = int(t_begin/timestep)
end_idx= int(t_end/timestep)

for name in ['Ep']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' DP backshift', linestyle='--')
    #plt.plot(ts_emt3ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_emt3ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' EMT')

## Generator terminal voltage

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Generator terminal voltage (V)')

for name in ['v_gen']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' DP backshift', linestyle='--')
    #plt.plot(ts_emt3ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' EMT')
plt.legend()
plt.show()

## Genrerator terminal Current

In [ ]:
plt.figure(figsize=(12,8))
plt.ylabel('Generator terminal current (A)')

for name in ['i_gen']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + 'SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' DP backshift', linestyle='--')
    #plt.plot(ts_emt3ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' EMT')

plt.legend()
plt.show()

## Voltage across line

In [ ]:
plt.figure(figsize=(12,8))

for name in ['v_line']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + 'SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' DP backshift', linestyle='--')
    #plt.plot(ts_emt3ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' EMT')
plt.legend()
plt.show()

## Current through line

In [ ]:
plt.figure(figsize=(12,8))

for name in ['i_line']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + 'SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).frequency_shift(60).values[begin_idx:end_idx], label=name + ' DP backshift', linestyle='--')
    #plt.plot(ts_emt3ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' EMT')
plt.legend()
plt.show()

## Generator electrical & mechanical energy

In [ ]:
plt.figure(figsize=(12,8))
plt.ylim(250e6,350e6)

for name in ['P_elec','P_mech']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' SP')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' DP', linestyle='--')
    #plt.plot(ts_emt3ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], np.sqrt(3/2)*ts_emt3ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' EMT')
plt.legend()
plt.show()

## Rotor frequency

In [ ]:
# plt.figure(figsize=(12,8))
# plt.xlabel('time (s)', fontsize=20)
# plt.ylabel('Rotor frequency (Hz)', fontsize=20)
# plt.xticks(fontsize=18)
# plt.yticks(fontsize=18)
# #plt.ylim(55,65)
# #plt.xlim(5.1,10)

# #if x_axis limits are changed above, change them again to consider the complete duration

# for name in ['wr_gen']:
#     plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*60/377, label='$\f_r$' + ' SP backshift')
#     plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*60/377, label='$\f_r$' + ' DP backshift', linestyle='--')
#     #plt.plot(ts_emt3ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_emt3ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' EMT')
# plt.legend(fontsize=18)
# #plt.title('Post-Fault',fontsize=20)
# plt.show()


In [ ]:
## Rotor angular velocity $\omega _r$

In [ ]:
plt.figure(figsize=(12,8))
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('Rotor angular velocity', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
#plt.ylim(360,400)
#fix y axis

for name in ['wr_gen']:
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label='$\omega _r$' + ' SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label='$\omega _r$' + ' DP backshift', linestyle='--')
    #plt.plot(ts_emt3ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx],ts_emt3ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' EMT')
plt.legend(fontsize=18)
#plt.title('Post-Fault',fontsize=20)
plt.show()


## Rotor angle $\delta _r$

In [ ]:
plt.figure(figsize=(12,8))
plt.xlabel('time (s)', fontsize=20)
#plt.ylim(-10,10)
plt.ylabel('Rotor angle in degrees', fontsize=20)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

for name in ['delta_r_gen']: 
    plt.plot(ts_sp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_sp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*180/3.14, label='$\delta _r$' + ' SP backshift')
    plt.plot(ts_dp1ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_dp1ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx]*180/3.14, label='$\delta _r$' + ' DP backshift', linestyle='--')
    #plt.plot(ts_emt3ph_TrStab_dl[name].interpolate(timestep).time[begin_idx:end_idx], ts_emt3ph_TrStab_dl[name].interpolate(timestep).values[begin_idx:end_idx], label=name + ' EMT')
    
plt.legend(fontsize=18)
#plt.title('Post-Fault',fontsize=20)
plt.show()


Frequency of load angle in 1hz-2hz range